In [ ]:
import pickle
import time
from collections import defaultdict

import matplotlib.pyplot as plt
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players as static_players
from nba_api.stats.endpoints import playercareerstats

players = pd.DataFrame(
    [
        static_players.find_players_by_full_name("Nikola Jokic")[0],
        static_players.find_players_by_full_name("Joel Embiid")[0],
        static_players.find_players_by_full_name("Giannis Antetokounmpo")[0],
        static_players.find_players_by_full_name("Luka Doncic")[0],
        static_players.find_players_by_full_name("Paul George")[0],
        static_players.find_players_by_full_name("James Harden")[0],
        static_players.find_players_by_full_name("Stephen Curry")[0],
        static_players.find_players_by_full_name("LeBron James")[0],
        static_players.find_players_by_full_name("Anthony Davis")[0],
        static_players.find_players_by_full_name("Damian Lillard")[0],
        static_players.find_players_by_full_name("Russell Westbrook")[0],
        static_players.find_players_by_full_name("Kevin Durant")[0],
        static_players.find_players_by_full_name("DeMar DeRozan")[0],
        static_players.find_players_by_full_name("LaMarcus Aldridge")[0],
        static_players.find_players_by_full_name("Jimmy Butler")[0],
        static_players.find_players_by_full_name("Kawhi Leonard")[0],
        static_players.find_players_by_full_name("Isaiah Thomas")[0],
        static_players.find_players_by_full_name("John Wall")[0],
        static_players.find_players_by_full_name("Chris Paul")[0],
        static_players.find_players_by_full_name("Draymond Green")[0],
        static_players.find_players_by_full_name("Kyle Lowry")[0],
        static_players.find_players_by_full_name("Marc Gasol")[0],
        static_players.find_players_by_full_name("Blake Griffin")[0],
        static_players.find_players_by_full_name("Tim Duncan")[0],
        static_players.find_players_by_full_name("Klay Thompson")[0],
        static_players.find_players_by_full_name("Jayson Tatum")[0],
        static_players.find_players_by_full_name("Shai Gilgeous-Alexander")[0],
        static_players.find_players_by_full_name("Donovan Mitchell")[0],
        static_players.find_players_by_full_name("Domantas Sabonis")[0],
        static_players.find_players_by_full_name("De'Aaron Fox")[0],
        static_players.find_players_by_full_name("Jalen Brunson")[0],
        static_players.find_players_by_full_name("Ja Morant")[0],
        static_players.find_players_by_full_name("Julius Randle")[0],
        static_players.find_players_by_full_name("Derrick Rose")[0],
        static_players.find_players_by_full_name("Rudy Gobert")[0],
        static_players.find_players_by_full_name("Ben Simmons")[0],
        static_players.find_players_by_full_name("Joakim Noah")[0],
        static_players.find_players_by_full_name("Devin Booker")[0],
    ]
)
players

mvp_vote_by_year = {
    23: ["Embiid", "Jokic", "Antetokounmpo", "Tatum", "Gilgeous-Alexander"],
    22: ["Jokic", "Embiid", "Antetokounmpo", "Booker", "Doncic"],
    21: ["Jokic", "Embiid", "Curry", "Antetokounmpo", "Paul"],
    20: ["Antetokounmpo", "James", "Harden", "Doncic", "Leonard"],
    19: ["Antetokounmpo", "Harden", "George", "Jokic", "Curry"],
    18: ["Harden", "James", "Davis", "Lillard", "Westbrook"],
    17: ["Westbrook", "Harden", "Leonard", "James", "Thomas"],
    16: ["Curry", "Leonard", "James", "Westbrook", "Durant"],
    15: ["Curry", "Harden", "James", "Westbrook", "Davis"],
    14: ["Durant", "James", "Griffin", "Noah", "Harden"],
}
mvp_vote_by_year

try:
    with open("game_logs.pkl", "rb") as fobj:
        game_logs = pickle.load(fobj)
except:
    game_logs = {}
    for year, last_names in mvp_vote_by_year.items():
        if year not in game_logs:
            game_logs[year] = {}
        season = f"20{year-1}-{year}"
        for last_name in last_names:
            player = players.query("last_name == @last_name").iloc[0]
            if player.id not in game_logs[year]:
                try:
                    gl = playergamelog.PlayerGameLog(player_id=player.id, season=season)
                    game_logs[year][player.id] = gl.get_data_frames()[0]
                except Exception as err:
                    print(err)
                time.sleep(1)
    
    with open("game_logs.pkl", "wb") as fobj:
        pickle.dump(game_logs, fobj, protocol=pickle.HIGHEST_PROTOCOL)

fig, ax = plt.subplots(figsize=(5.5, 3), dpi=120)
for year, subinfos in infos.groupby("year"):
    for row, info in subinfos.iterrows():
        if info.place > 3:
            continue
        xoffset = +info.place / 8- 0.5 / 2
        ax.plot(
            year + xoffset,
            info.wins,
            "o",
            color="k",
            mew=0.5,
            ms=2,
            mfc="none",
            alpha=0.2,
        )
        ax.text(year + xoffset, info.wins, f"{info.place}", ha="center", va="center", size=8)
ax.set_xticks(np.arange(14, 24))
ax.grid(axis="y", color=[0.9]*3)
ax.set_ylabel("Wins")
ax.set_xlabel("Year of vote")
ax.spines[["top", "right"]].set_visible(False)
fig.tight_layout()

infos = []
for year, logs in game_logs.items():
    for place, (player_id, df) in enumerate(logs.items(), start=1):
        player = players[players.id == player_id].iloc[0]
        ws = len(df.query("WL == 'W'"))
        ls = len(df.query("WL == 'L'"))
        infos.append(
            {
                "year": year,
                "full_name": player.full_name,
                "place": place,
                "wins": ws,
                "losses": ls,
                "gp": ws + ls,
                "wpct": ws / (ws + ls),
            }
        )
infos = pd.DataFrame(infos)
infos

wins = []
for yr in range(15, 24):
    season = f"20{yr-1}-{yr}"
    for idx, player in players.iterrows():
        gl = playergamelog.PlayerGameLog(player_id=player.id, season="2022-23")
        gldf = gl.get_data_frames()[0]
        w = len(gldf.query("WL == 'W'"))
        l = len(gldf.query("WL == 'L'"))
        wins.append({"year": yr, "id": player.id, "wins": w, "losses": l, "gp": w + l})
wins = pd.DataFrame(wins)
wins